In [1]:
import pandas as pd

In [3]:
base_ = pd.read_csv('base_name.csv').dropna()
main_ = pd.read_csv('contributor_name.csv').dropna()


In [4]:
main_

,index,contributor_name
0,0,actblue
1,1,NATIONAL REPUBLICAN CONGRESSIONAL COMMITTEE EX...
2,2,working america
3,3,DEMOCRATIC CONGRESSIONAL CAMPAIGN COMMITTEE
4,4,act blue
5,5,at t
6,6,democratic congressional campaign
7,7,REALTORS POLITICAL ACTION COMMITTEE
8,8,DNC SERVICES CORPORATION/DEMOCRATIC NATIONAL C...
9,9,UPSPAC
